# Toying with my Algorithm

In [33]:
from splitwise import OptimalSplit

In [34]:
transactions = [["Angela", "Olaf", 10],
    ["Olaf", "Friedrich", 5],
    ["Friedrich", "Angela", 5],
    ["Friedrich", "Barbie", 10],
    ["Barbie", "Olaf", 7],
    ["Ken", "Barbie", 5],
    ["Barbie", "Ken", 5]]

splitter = OptimalSplit()
txns = splitter.minTransfers(transactions)

print(f"{len(txns)} payments need to be made.")

for debtor, creditor, amount in txns:
    print(f"{debtor} pays {creditor} {amount}")






defaultdict(<class 'int'>, {'Angela': 5, 'Olaf': -12, 'Friedrich': 10, 'Barbie': -3, 'Ken': 0})
{'Angela': 5, 'Olaf': -12, 'Friedrich': 10, 'Barbie': -3}
3 payments need to be made.
Olaf pays Angela 5
Olaf pays Friedrich 7
Barbie pays Friedrich 3
